# 1. Introduction / Business Problem

In this project, I will use Foursquare data to determine the best neighboorhood in the Bronx to open a new pizza restaruant based on the similarities to neighboorhoods with a high density of pizza restaurants. This analysis would be useful to anyone thinking about opening a pizza restaurant in the Bronx; it will help them determine what areas are likely to be successful and not already saturated with pizza restaraunts. 

# 2. Data

For this project, I will be using the New York data set we used in week 3 along with Foursquare data.

First, I'll download the packages I'll need and the NY data.

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [8]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    


Data downloaded!


In [11]:
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### I'm going to focus my analysis on Bronx.

In [12]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Now, I'll use Foursquare to get information on the neighborhoods in Bronx.

In [13]:
CLIENT_ID = 'BHCKRM3W0KWRV3BG0KPBCAJCTKKKKK1ECTGROD5N033CAFUI' # your Foursquare ID
CLIENT_SECRET = 'HMWK5JCGN3SCEHZRLLIR2JUGTBS4A3KPAZQG5SSXSF3SGBA2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [19]:
neighborhood_latitude = bronx_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = bronx_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = bronx_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [21]:
results = requests.get(url).json()

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Walgreens,Pharmacy,40.896528,-73.844700
2,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568
3,Rite Aid,Pharmacy,40.896649,-73.844846
4,Dunkin',Donut Shop,40.890459,-73.849089


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [26]:
print(bronx_venues.shape)
bronx_venues.head()

(1221, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


# 3. Methodology

Now that I have my data ready, I will start my analysis. 

First, I'll one hot encode the data and sort to see if there are neighborhoods that have a greater concentration of pizza places.

In [27]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Supply Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Multiplex,Music Venue,Nightclub,Office,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Post Office,Pub,Recreation Center,Rental Car Location,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [29]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Supply Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Multiplex,Music Venue,Nightclub,Office,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Post Office,Pub,Recreation Center,Rental Car Location,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Allerton,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.068966,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.034483,0.068966,0.034483,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.000000,0.034483,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.137931,0.034483,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0

In [30]:
num_top_venues = 5

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                venue  freq
0         Pizza Place  0.14
1    Department Store  0.07
2         Supermarket  0.07
3  Chinese Restaurant  0.07
4          Donut Shop  0.03


----Baychester----
               venue  freq
0         Donut Shop  0.10
1  Accessories Store  0.05
2        Music Venue  0.05
3     Sandwich Place  0.05
4  Electronics Store  0.05


----Bedford Park----
                 venue  freq
0          Pizza Place  0.11
1                Diner  0.11
2   Chinese Restaurant  0.08
3   Mexican Restaurant  0.08
4  Fried Chicken Joint  0.06


----Belmont----
                venue  freq
0  Italian Restaurant  0.19
1         Pizza Place  0.08
2       Deli / Bodega  0.07
3              Bakery  0.05
4       Grocery Store  0.03


----Bronxdale----
                         venue  freq
0                         Bank  0.08
1        Performing Arts Venue  0.08
2           Italian Restaurant  0.08
3               Breakfast Spot  0.08
4  Eastern European Restaurant  0.08


----C

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [100]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Chinese Restaurant,Department Store,Supermarket,Check Cashing Service,Spanish Restaurant,Fast Food Restaurant,Bus Station,Electronics Store,Donut Shop
1,Baychester,Donut Shop,Accessories Store,Supermarket,Fried Chicken Joint,Mattress Store,Men's Store,Mexican Restaurant,Music Venue,Fast Food Restaurant,Electronics Store
2,Bedford Park,Pizza Place,Diner,Chinese Restaurant,Mexican Restaurant,Sandwich Place,Fried Chicken Joint,Deli / Bodega,Italian Restaurant,Pharmacy,Coffee Shop
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Bank,Grocery Store,Dessert Shop,Donut Shop,Shoe Store,Coffee Shop
4,Bronxdale,Italian Restaurant,Bank,Pizza Place,Chinese Restaurant,Performing Arts Venue,Paper / Office Supplies Store,Eastern European Restaurant,Breakfast Spot,Mexican Restaurant,Supermarket


#### Now that I can see there are neighborhoods with a higher density of pizza places, i.e. it is the most common venue, I will use K Means clustering to see what neigborhoods are most like each other.

In [57]:
# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 1, 0, 4, 7, 5, 7], dtype=int32)

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0,Pharmacy,Deli / Bodega,Ice Cream Shop,Dessert Shop,Pizza Place,Sandwich Place,Laundromat,Gas Station,Donut Shop,Electronics Store
1,Bronx,Co-op City,40.874294,-73.829939,7,Bus Station,Baseball Field,Post Office,Chinese Restaurant,Fast Food Restaurant,Basketball Court,Bagel Shop,Grocery Store,Park,Discount Store
2,Bronx,Eastchester,40.887556,-73.827806,2,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Donut Shop,Bowling Alley,Fast Food Restaurant,Metro Station,Seafood Restaurant,Bakery
3,Bronx,Fieldston,40.895437,-73.905643,2,Cosmetics Shop,River,Plaza,Bus Station,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farmers Market
4,Bronx,Riverdale,40.890834,-73.912585,2,Park,Bus Station,Gym,Medical Supply Store,Baseball Field,Bank,Food Truck,Plaza,Dive Bar,Doctor's Office


In [48]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### I don't think five groups was enough to give me meaningful analysis as there are clearly more in one of the groups. I'm going to try again with 10 clusters.

In [62]:
# set number of clusters
kclusters = 10

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans2.labels_[0:10] 

array([4, 4, 4, 4, 1, 0, 4, 7, 5, 7], dtype=int32)

In [101]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans2.labels_)

bronx_merged = bronx_data

bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0,Pharmacy,Deli / Bodega,Ice Cream Shop,Dessert Shop,Pizza Place,Sandwich Place,Laundromat,Gas Station,Donut Shop,Electronics Store
1,Bronx,Co-op City,40.874294,-73.829939,7,Bus Station,Baseball Field,Post Office,Chinese Restaurant,Fast Food Restaurant,Basketball Court,Bagel Shop,Grocery Store,Park,Discount Store
2,Bronx,Eastchester,40.887556,-73.827806,2,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Donut Shop,Bowling Alley,Fast Food Restaurant,Metro Station,Seafood Restaurant,Bakery
3,Bronx,Fieldston,40.895437,-73.905643,2,Cosmetics Shop,River,Plaza,Bus Station,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farmers Market
4,Bronx,Riverdale,40.890834,-73.912585,2,Park,Bus Station,Gym,Medical Supply Store,Baseball Field,Bank,Food Truck,Plaza,Dive Bar,Doctor's Office


In [102]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 4. Results

#### Now, let's see which cluster has the most pizza places at their most common venue.

In [126]:
bronx_merged.loc[bronx_merged['1st Most Common Venue'] == 'Pizza Place', bronx_merged.columns[[4] + list(range(5, bronx_merged.shape[1]))]]

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,4,Pizza Place,Bar,Supermarket,Bakery,Mexican Restaurant,Latin American Restaurant,Sandwich Place,Fried Chicken Joint,Spanish Restaurant,Pharmacy
7,4,Pizza Place,Park,Bank,Chinese Restaurant,Pharmacy,Fast Food Restaurant,Bus Station,Burger Joint,Mexican Restaurant,Mobile Phone Shop
12,4,Pizza Place,Diner,Chinese Restaurant,Mexican Restaurant,Sandwich Place,Fried Chicken Joint,Deli / Bodega,Italian Restaurant,Pharmacy,Coffee Shop
13,4,Pizza Place,Grocery Store,Bakery,Fast Food Restaurant,Shoe Store,Fried Chicken Joint,Supermarket,Sandwich Place,Donut Shop,Bus Station
16,4,Pizza Place,Lounge,Discount Store,Restaurant,Paella Restaurant,Donut Shop,Café,Shoe Store,Bus Stop,Fast Food Restaurant
19,4,Pizza Place,Pharmacy,Sandwich Place,Gym / Fitness Center,Grocery Store,Mexican Restaurant,Martial Arts Dojo,Diner,Clothing Store,Discount Store
31,4,Pizza Place,Deli / Bodega,Middle Eastern Restaurant,Spa,Hookah Bar,Diner,Coffee Shop,Film Studio,BBQ Joint,Donut Shop
32,4,Pizza Place,Deli / Bodega,Burger Joint,Bakery,Italian Restaurant,Bank,Buffet,Pharmacy,Spa,Spanish Restaurant
35,4,Pizza Place,Bank,Chinese Restaurant,Italian Restaurant,Social Club,Bar,Burger Joint,Food,Mexican Restaurant,Sandwich Place
50,4,Pizza Place,Chinese Restaurant,Department Store,Supermarket,Check Cashing Service,Spanish Restaurant,Fast Food Restaurant,Bus Station,Electronics Store,Donut Shop


#### All of the neigborhoods with pizza places as their most common venue are in cluster 4, so let's look at cluster 4 more closely.

In [107]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 4, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kingsbridge,Pizza Place,Bar,Supermarket,Bakery,Mexican Restaurant,Latin American Restaurant,Sandwich Place,Fried Chicken Joint,Spanish Restaurant,Pharmacy
6,Woodlawn,Pub,Food Truck,Deli / Bodega,Pizza Place,Playground,Bar,Grocery Store,Food & Drink Shop,Donut Shop,Park
7,Norwood,Pizza Place,Park,Bank,Chinese Restaurant,Pharmacy,Fast Food Restaurant,Bus Station,Burger Joint,Mexican Restaurant,Mobile Phone Shop
9,Baychester,Donut Shop,Accessories Store,Supermarket,Fried Chicken Joint,Mattress Store,Men's Store,Mexican Restaurant,Music Venue,Fast Food Restaurant,Electronics Store
10,Pelham Parkway,Italian Restaurant,Pizza Place,Bank,Ice Cream Shop,Coffee Shop,Chinese Restaurant,Performing Arts Venue,Sandwich Place,Donut Shop,Bus Station
11,City Island,Boat or Ferry,Thrift / Vintage Store,Seafood Restaurant,Grocery Store,Spanish Restaurant,Smoke Shop,Pizza Place,Deli / Bodega,Pharmacy,Italian Restaurant
12,Bedford Park,Pizza Place,Diner,Chinese Restaurant,Mexican Restaurant,Sandwich Place,Fried Chicken Joint,Deli / Bodega,Italian Restaurant,Pharmacy,Coffee Shop
13,University Heights,Pizza Place,Grocery Store,Bakery,Fast Food Restaurant,Shoe Store,Fried Chicken Joint,Supermarket,Sandwich Place,Donut Shop,Bus Station
14,Morris Heights,Recreation Center,Grocery Store,Spanish Restaurant,Supermarket,Bus Station,Bank,Food,Pizza Place,Deli / Bodega,Pharmacy
15,Fordham,Mobile Phone Shop,Fast Food Restaurant,Donut Shop,Shoe Store,Bank,Deli / Bodega,Pharmacy,Supplement Shop,Clothing Store,Gym / Fitness Center


#### Let's see which neighborhoods have a pizza place as their least common venue to decide where would be an ideal spot to start a new one.

In [156]:
bronx_merged.loc[bronx_merged['10th Most Common Venue'] == 'Pizza Place', bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [157]:
bronx_merged.loc[bronx_merged['9th Most Common Venue'] == 'Pizza Place', bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [158]:
bronx_merged.loc[bronx_merged['8th Most Common Venue'] == 'Pizza Place', bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Morris Heights,Recreation Center,Grocery Store,Spanish Restaurant,Supermarket,Bus Station,Bank,Food,Pizza Place,Deli / Bodega,Pharmacy


In [159]:
bronx_merged.loc[bronx_merged['7th Most Common Venue'] == 'Pizza Place', bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,City Island,Boat or Ferry,Thrift / Vintage Store,Seafood Restaurant,Grocery Store,Spanish Restaurant,Smoke Shop,Pizza Place,Deli / Bodega,Pharmacy,Italian Restaurant
43,Unionport,Ice Cream Shop,Latin American Restaurant,Donut Shop,Spanish Restaurant,Deli / Bodega,Dance Studio,Pizza Place,Diner,Comfort Food Restaurant,Pet Store


# 5. Discussion

From the analysis above, Morris Heights is the ideal neighborhood to open a new pizza place. It is clustered with other neighorhoods that have pizza places as their most common venue, yet pizza places are their 8th most common venue. This indicates that the pizza market is not crowded in Morris Heights and an additional pizza place would likely do very well.

City Island and Unionport are also attractive candidates as they are in the same cluster and only have pizza places as their 7th most common venue.

# 6. Conclusion

Opening a new restaruant is always a risk. Chosing the right location can make a big difference on the restaruant's financial success or failure. In this this project, I used Foursquare location data to determine the best neighboorhood in the Bronx to open a new pizza place by clustering neighborhoods, identifying the cluster with the most pizza places as their most common venue, and determining which neigborhoods within that cluster had pizza places as their least common venues. The neighborhoods within the cluster with the pizza venues as their least common venues can likely support additional pizza places. 

This analysis would be useful to anyone thinking about opening a pizza restaurant in the Bronx; it indicates that Morris Heights is the ideal neighborhood in the Bronx to open a new pizza place. Morris Heights is clustered with other neighorhoods that have pizza places as their most common venue, yet pizza places are their 8th most common venue. This indicates that the pizza market is not crowded in Morris Heights and an additional pizza place would likely do very well.

Additional analysis including demographic data, for example average age and income in each neighborhood, as well as financial data such as average rent and business operating costs in each neighborhood would also be useful in making the decision on where to open a new pizza place in the Bronx.